In [36]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
train_tape = "Z:/rppg/ccnu_datatape_450x8x8_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_450x8x8_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)
train, valid = to_tf(train).cache(), to_tf(valid).cache()
#train, valid = to_tf(train).cache(f'{tmp}/ours'), to_tf(valid).cache(f'{tmp}/ours')
#train, valid = to_tf(train), to_tf(valid)

In [2]:
m1 = M_1()
m1.compile(optimizer='Nadam', loss='mae')
m1.build(input_shape=(None, 450, 8, 8, 3))
print(f'Flops per frame:{get_flops(m1, input_sig=[tf.TensorSpec([1, 450, 8, 8, 3])])/450:.0f}')
m1.summary()

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Flops per frame:260557
Model: "m_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 450, 64)           12352     
_________________________________________________________________
spectral_transform (Spectral multiple                  82560     
_________________________________________________________________
conv1d_2 (Conv1D)            multiple                  41024     
_________________________________________________________________
sequential_2 (Sequential)    (None, 450, 64)           256       
_________________________________________________________________
spectral_transform_1 (Spectr multiple                  49792     
_________________________________________________________________
conv1d_4 (Conv1D)            multiple                  10272     
_________________

In [51]:
m1.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/m1.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
m1.load_weights('../weights/m1.h5')

Epoch 1/10
1410/1410 [==============================] - 24s 16ms/step - loss: 0.6872 - val_loss: 0.6440

Epoch 00001: val_loss improved from inf to 0.64403, saving model to ../weights\m1.h5
Epoch 2/10
1410/1410 [==============================] - 23s 16ms/step - loss: 0.5454 - val_loss: 0.6066

Epoch 00002: val_loss improved from 0.64403 to 0.60655, saving model to ../weights\m1.h5
Epoch 3/10
1410/1410 [==============================] - 23s 16ms/step - loss: 0.4962 - val_loss: 0.5684

Epoch 00003: val_loss improved from 0.60655 to 0.56843, saving model to ../weights\m1.h5
Epoch 4/10
1410/1410 [==============================] - 23s 16ms/step - loss: 0.4632 - val_loss: 0.5557

Epoch 00004: val_loss improved from 0.56843 to 0.55574, saving model to ../weights\m1.h5
Epoch 5/10
1410/1410 [==============================] - 23s 16ms/step - loss: 0.4466 - val_loss: 0.5736

Epoch 00005: val_loss did not improve from 0.55574
Epoch 6/10
1410/1410 [==============================] - 23s 16ms/step - 

In [38]:
m1.load_weights('../weights/m1.h5')

In [62]:
#eval_on_dataset(test_set_CCNU, m1, 450, (8, 8), step=5, batch=32, save='../results/ours_CCNU_CCNU.h5')
get_metrics('../results/ours_CCNU_CCNU.h5',)

100%|██████████| 179/179 [00:05<00:00, 33.21it/s]


{'Sliding window': {'MAE': 1.032, 'RMSE': 3.784, 'R': 0.93091},
 'Whole video': {'MAE': 0.516, 'RMSE': 1.22, 'R': 0.99178}}

In [5]:
#eval_on_dataset(test_set_CCNU_rPPG, m1, 450, (8, 8), step=5, batch=32, save='../results/ours_CCNU_CCNU_rPPG.h5')
get_metrics('../results/ours_CCNU_CCNU_rPPG.h5',)
#get_metrics_HRV('../results/ours_CCNU_CCNU_rPPG.h5',)

{'Sliding window': {'MAE': 0.909, 'RMSE': 3.393, 'R': 0.93793},
 'Whole video': {'MAE': 0.654, 'RMSE': 1.951, 'R': 0.97916}}

In [8]:
eval_on_dataset(test_set_PURE, m1, 450, (8, 8), step=5, batch=32, save='../results/ours_CCNU_PURE.h5')
get_metrics('../results/ours_CCNU_PURE.h5',)
#get_metrics_HRV('../results/ours_CCNU_PURE.h5',)

100%|██████████| 59/59 [00:01<00:00, 40.56it/s]


{'Sliding window': {'MAE': 0.359, 'RMSE': 1.136, 'R': 0.99881},
 'Whole video': {'MAE': 0.338, 'RMSE': 0.52, 'R': 0.99978}}

In [6]:
eval_on_dataset(test_set_UBFC_rPPG2, m1, 450, (8, 8), step=5, batch=32, save='../results/ours_CCNU_UBFC.h5')
get_metrics('../results/ours_CCNU_UBFC.h5',)
#get_metrics_HRV('../results/ours_CCNU_UBFC.h5',)

100%|██████████| 42/42 [00:00<00:00, 42.53it/s]


{'Sliding window': {'MAE': 1.296, 'RMSE': 3.567, 'R': 0.97857},
 'Whole video': {'MAE': 0.921, 'RMSE': 1.439, 'R': 0.99679}}

In [34]:
#eval_on_dataset(test_set_MMPD, m1, 450, (8, 8), step=10, batch=32, save='../results/ours_CCNU_MMPD.h5')
get_metrics('../results/ours_CCNU_MMPD.h5', motion='Stationary', skin_color='3', light=['LED-high', 'LED-low', 'Incandescent',])

{'Sliding window': {'MAE': 2.681, 'RMSE': 7.381, 'R': 0.69042},
 'Whole video': {'MAE': 1.512, 'RMSE': 3.93, 'R': 0.91478}}

In [40]:
eval_on_dataset(test_set_COHFACE, m1, 450, (8, 8), step=1, batch=32, save='../results/ours_CCNU_COHFACE.h5')
get_metrics('../results/ours_CCNU_COHFACE.h5')

100%|██████████| 164/164 [00:06<00:00, 23.66it/s]


{'Sliding window': {'MAE': 17.117, 'RMSE': 26.319, 'R': -0.10291},
 'Whole video': {'MAE': 16.051, 'RMSE': 25.736, 'R': -0.11651}}